In [47]:
import os
import urllib
import json
import pandas as pd
from pandas import json_normalize
from datetime import datetime
from datetime import date
import time

In [61]:
# GitHub API unauthenticated requests rate limit = 10 requests per minute.
orgs = ["nhsx","111Online","NHSDigital","nhsconnect","nhsengland"]

page = 1
empty = []
while data is not empty:
    page = page + 1
    url = "https://api.github.com/orgs/"+str(orgs[0])+"/repos?page="+str(page)+"&per_page=100"
    print(url)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    time.sleep(61) # Sleep for 61 seconds

https://api.github.com/orgs/nhsx/repos?page=2&per_page=100


HTTPError: HTTP Error 403: rate limit exceeded

In [49]:
# last modified datex
data_updated = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print(data_updated)

31/03/2021 11:38:03


In [50]:
with open('github_api/nhsx_repos.json') as f:
  data = json.load(f)

In [62]:
df = pd.DataFrame()
for org in orgs:
    url = "https://api.github.com/orgs/"+str(org)+"/repos?page=1&per_page=100"
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    flat_data = json_normalize(data)
    df = df.append(flat_data)
df.columns = df.columns.str.replace('.', '_')
df.head()

HTTPError: HTTP Error 403: rate limit exceeded

In [63]:
# Group the data frame by month and item and extract a number of stats from each group
df_group = df.groupby(
    ['owner_login']
).agg(
    {
        # find the number of open repos
        'name': "count",
        'size': "sum",
        'stargazers_count': "sum",
        'forks_count': "sum",
        'open_issues_count': "sum",
        'license_name': lambda x:x.value_counts().index[0],
        'language': lambda x:x.value_counts().index[0]
    }
)
df_group

KeyError: 'owner_login'

In [53]:
df_html = df_group.head(10).to_html(classes='summary')

In [54]:
# Write HTML String to file.html
with open("_includes/table1.html", "w") as file:
    file.write(df_html)